In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import urllib.request
from bs4 import BeautifulSoup

In [2]:
def substringBetween(string,sub1,sub2):
    sub = string.split(sub1,1)[1]
    sub = sub.split(sub2,1)[0]
    return sub.strip()
    
class Points:
    def __init__(self,points,level):
        self.points = points
        self.level = level

In [3]:
url = 'https://www.wtatennis.com/players/player/318033/title/ashleigh-barty-0'
html = urllib.request.urlopen(url)
soup = BeautifulSoup(html, 'lxml')

In [4]:
columns = ["Tournament","Level","Surface",
           "Rank","Seed","Round",
           "Opponent Name","Opponent Rank",           
           "Opponent Seed","Opponent Country",
           "Score","Result"]

table = []
points_ytd = []

singles = soup.find('div',{'class': "matchtype-singles"})
matches = singles.findAll('div',{'class': "daySeq wta-table-data data-player-matches"})
wins = 0
losses = 0
for i in range(len(matches)):
    string = matches[i].text
    origstring = string
    print(string+"\n")
    tournament = string.split(",",1)[0].strip()
    level = substringBetween(string,'Level:','Prize')
    surface = substringBetween(string,'Surface:','Draw')
    rank = substringBetween(string,'Rank:','Seed')
    seed = substringBetween(string,'Seed Entry:','RoundResult')
    string = string.split("Scores",1)[1].strip()
    while True:
        for i in range(15):
            if string[i]=='W' or string[i]=='L':
                result = string[i]
                break
        round_ = string.split(result,1)[0].strip()
        string = string.split(round_,1)[1].strip()
        if string[:5]!='W BYE':
            opponent_name = substringBetween(string,result,'(')
            string = string.split(opponent_name,1)[1].strip()
            opponent_country = substringBetween(string,'(',')')
            string = string.split(")",1)[1].strip()
            opponent_rank = string.split(" ",1)[0].strip()
            opponent_seed = ''
            if opponent_name[0]=='[':
                opponent_seed = substringBetween(opponent_name,'[',']')
                opponent_name = opponent_name.split(']',1)[1].strip()
            string = string.split(opponent_rank,1)[1].strip()
            score = ''
            letters = 0
            letterstring = ''
            while letters<3:
                if not string[0].isalpha():
                    score = score+string[0]
                else:
                    letters+=1
                    letterstring = letterstring+string[0]
                string = string[1:]
            if letterstring=='RET':
                score = score+letterstring
            else:
                string = letterstring+string
            score = score.strip()
            if result=='W':
                wins+=1
            else:
                losses+=1
        else:
            opponent_name = 'BYE'
            opponent_country = ''
            opponent_rank = ''
            opponent_seed = ''
            score = ''
            result = ''
            string = string.split('BYE - -',1)[1].strip()
        match = [tournament,level,surface,rank,
                 seed,round_,opponent_name,
                 opponent_rank,opponent_seed,
                 opponent_country,score,result]
        #print(match)
        table.append(match)
        string = string.strip()
        if string[:3]=='WTA':
            break
    points = int(substringBetween(origstring,'+','Prize'))
    points_ytd.append(Points(points,level))
    
    
df = pd.DataFrame(data=table,columns=columns)

print(df)
print("\nPlayer record: "+str(wins)+"-"+str(losses))   
for i in range(len(points_ytd)):
    if i==len(points_ytd)-1:
        print(points_ytd[i].points)
        break
    print(points_ytd[i].points, end='+')

   BIRMINGHAM - BIRMINGHAM, GREAT BRITAIN   Date:  June 17, 2019 Level:   Premier  Prize Money:  $941,163 Surface:  GRASS Draw:  32M/32Q/16D Rank:  2 Seed Entry:  2RoundResultOpponentRankScores Round 32W Donna Vekic (CRO) 22 6-3 6-4 Round 16W Jennifer Brady (USA) 66 6-3 6-1 QuarterfinalsW [W] Venus Williams (USA) 55 6-4 6-3 SemifinalsW Barbora Strycova (CZE) 51 6-4 6-4 FinalW [8] Julia Goerges (GER) 19 6-3 7-5 WTA Ranking Points: +470  Prize Money: $176,215  

  ROLAND GARROS , FRANCE  Date:  May 26, 2019 Level:   Grand Slam  Prize Money:  $23,029,029 Surface:  CLAY Draw:  Rank:  8 Seed Entry:  8RoundResultOpponentRankScores Round 128W Jessica Pegula (USA) 72 6-3 6-3 Round 64W Danielle Collins (USA) 36 7-5 6-1 Round 32W Andrea Petkovic (GER) 69 6-3 6-1 Round 16W Sofia Kenin (USA) 35 6-3 3-6 6-0 QuarterfinalsW [14] Madison Keys (USA) 14 6-3 7-5 SemifinalsW Amanda Anisimova (USA) 51 6-7(4) 6-3 6-3 FinalW Marketa Vondrousova (CZE) 38 6-1 6-3 WTA Ranking Points: +2000  Prize Money: $2,649,